In [1]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt

--2022-05-03 01:51:23--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  18.7MB/s    in 6m 39s  

2022-05-03 01:58:02 (16.1 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-05-03 01:58:51--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [2]:
import tensorflow as tf
import numpy as np

model = tf.keras.applications.EfficientNetB4(weights='imagenet')

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')

validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
for i in range(len(mapping_list_from)):
  mapping_list_to.append(mapping_list_from[i].split(" ")[1])
  mapping_list_from[i] = mapping_list_from[i].split(" ")[0]

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

78880768/78864416 [==============================] - 1s 0us/step


In [3]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(380, 380))
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  preds = model.predict(x)

  id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=1)[0][0][0]
  index = mapping_list_from.index(id)

  if mapping_list_to[index] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=5)[0][ii][0]
      index = mapping_list_from.index(id)
      if mapping_list_to[index] == validation_list[i]:
        top_5 = top_5 + 1
        break

  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))

accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

49152/35363 [=========================================] - 0s 0us/step
1%, Top 1:0.802 Top 5:0.962
2%, Top 1:0.806 Top 5:0.964
3%, Top 1:0.826 Top 5:0.9653333333333334
4%, Top 1:0.819 Top 5:0.961
5%, Top 1:0.818 Top 5:0.962
6%, Top 1:0.816 Top 5:0.9616666666666667
7%, Top 1:0.814 Top 5:0.9608571428571429
8%, Top 1:0.81375 Top 5:0.9575
9%, Top 1:0.8168888888888889 Top 5:0.9568888888888889
10%, Top 1:0.817 Top 5:0.955
11%, Top 1:0.8185454545454546 Top 5:0.9549090909090909
12%, Top 1:0.8176666666666667 Top 5:0.956
13%, Top 1:0.82 Top 5:0.956
14%, Top 1:0.8185714285714286 Top 5:0.9562857142857143
15%, Top 1:0.8208 Top 5:0.9562666666666667
16%, Top 1:0.819875 Top 5:0.956
17%, Top 1:0.8211764705882353 Top 5:0.956
18%, Top 1:0.8196666666666667 Top 5:0.9561111111111111
19%, Top 1:0.8202105263157895 Top 5:0.9563157894736842
20%, Top 1:0.82 Top 5:0.9558
21%, Top 1:0.8194285714285714 Top 5:0.956
22%, Top 1:0.8181818181818182 Top 5:0.9553636363636364
23%, Top 1:0.818 Top 5:0.9554782608695652
24%, T